In [5]:
import pandas as pd
import pandas
import seaborn as sns
import pyodbc
from gensim import corpora, models, similarities
import nltk
import re
import random
import binascii
import logging, gensim, bz2
from nltk.tag import pos_tag
import logging
import re
import tempfile
import tarfile, zipfile
from gensim.utils import tokenize
from collections import defaultdict
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import vertica_python

tenant_id = 289
unuseful_list = 'with been him at am but not so for a of the and to in is very i my on it was as he his her you your we are have she had they us this hello hi over that me';
input_option = ['industry','tenant id','tenant name'];

query= """
SELECT id, comment FROM r4e_mongo.ratings 
WHERE 
    industry in ('health-care-physicians') 
AND location_name not ilike '%physician%' 
AND location_name not ilike '%clinic%' 
AND location_name not ilike '%emergency%' 
AND location_name not ilike '%center%' 
AND location_name not ilike '%health%' 
AND location_name not ilike '%massage%' 
AND location_name not ilike '%spa%' 
AND comment not ilike '%dental%' 
AND comment not ilike '%dentist%' 
AND location_name not ilike '%dental%' 
AND location_name not ilike '%wellness%' 
AND location_name not ilike '%wellness%' 
AND comment is not null and length(comment) > 50 
AND tenant_id not in ('4634', '3054') 
AND tenant_type not in ('demo') 
ORDER BY RANDOM() LIMIT 50000
"""



frequency = defaultdict(int)

conn_info = {'host': 'vertica04.reputation.ec2', 
             'port': 5433, 
             'user': 'rep_engine_user', 
             'password': 'r3p_3ng1n3', 
             'database': 'rwarehouse'}

cnxn = vertica_python.connect(**conn_info)

%matplotlib inline
sample_size = 10000

In [6]:
from IPython.html import widgets
from IPython.html.widgets import interact
from IPython.display import display

#input_type = widgets.RadioButtons(description="Select a type of input: ", options= input_option)
#input_content = widgets.Text(description='Enter input: ', width = 250);                         

#container = widgets.Box(children = [input_type, input_content]);
#display(container);


In [7]:
#constraint = "";

#if input_type.value == 'industry':
#    constraint = " AND tenants.industry like '%" + str(input_content.value) +"%'";
#elif input_type.value == 'tenant id':
#    constraint = " AND locations.tenant_id = " + str(input_content.value);
#else: 
#    constraint = " AND UPPER(tenants.name) like UPPER('%" + str(input_content.value) + "%')";

#query = query_temp + constraint + " LIMIT 50000";

print(query)


SELECT id, comment FROM r4e_mongo.ratings 
WHERE 
    industry in ('health-care-physicians') 
AND location_name not ilike '%physician%' 
AND location_name not ilike '%clinic%' 
AND location_name not ilike '%emergency%' 
AND location_name not ilike '%center%' 
AND location_name not ilike '%health%' 
AND location_name not ilike '%massage%' 
AND location_name not ilike '%spa%' 
AND comment not ilike '%dental%' 
AND comment not ilike '%dentist%' 
AND location_name not ilike '%dental%' 
AND location_name not ilike '%wellness%' 
AND location_name not ilike '%wellness%' 
AND comment is not null and length(comment) > 50 
AND tenant_id not in ('4634', '3054') 
AND tenant_type not in ('demo') 
ORDER BY RANDOM() LIMIT 50000



In [8]:

result = pd.read_sql(query, cnxn)

#result = pd.read_csv("/Users/zzhao/Documents/Anthony_code_and_review/a894359.csv")


result

,id,comment
0,VITALS.b3UfGls-LzDD8hWhQ7lRJA.dzxTRcdPcOukfhSd...,This is one of the best Drs. in the world ! Ye...
1,VITALS.6k9crmezPz9_hCZRuPxV_g.gnuy6VinKQVP2qIH...,Dr Wong has been seeing to my care for over 7 ...
2,VITALS.kA8YcH1Vzee8QLLu9G0QbQ.t0ieJOmsu31TDYRG...,Dr. Corzilius is an amazing dr and an amazing ...
3,VITALS.SYdMddyG2flCcho4QpaApA.09CPHCp-QLrOuvKL...,"Very bright, conscientious, and friendly. He g..."
4,JUDYS_BOOK.BJcHUqV_Yr-nImCKXUc_3w.16854903,Recommend highly for Dr. White's area of medic...
5,VITALS.GH4isHvFhtS9GAySnrbEdA.s7TmS7rQscIWSjqY...,dr d is the best dr around shereally listens t...
6,YELP.6ksxR3COBbKLMEARd5S_8Q.to4kes8MRViPNDY8Vr...,As much as I tried to find a good experience r...
7,RATE_MDS.gjRhhrixgVhS7b0_jXnOug.K6thxLCOzXfU_2...,We went to him with a perfectly healthy 5 year...
8,KIOSK_dae6e710a214ce094db24a67d164f9f8_1787ba9...,Dr. Lauer is fantastic! He works well with my...
9,YELP.lRfFarL3ioxY0zHNU0l_Aw.7r4zhr7cs1JDbujhSQ...,I've been seeing Dr. Durr for over 7 years. I...


In [9]:
#type(result)
result.columns

Index([u'id', u'comment'], dtype='object')

In [10]:
#comments = result['comment'].values;
#ratings = result['normalized_rating'].values;
id_list = result['id'].values.tolist();
comment_list = result['comment'].values.tolist();
#comment_list
#for counter in range(len(comments)):
#    comment_list.append([comments[counter], ratings[counter]]);

type(comment_list[0])

unicode

In [11]:
stoplist = set(unuseful_list.split());


for indi_comment in comment_list:
    if(len(re.split(r'[;,.!*?~+\r\n\s\' ]\s*', indi_comment.lower())) < 5):
        #print(comment_list[i])
        comment_list.remove(indi_comment);
        
filtered_comment = [[word for word in re.split(r'[;,.!*?~+\r\n\s\' ]\s*', comment.lower()) if word not in stoplist]
      for comment in comment_list]

#print("ori comments: " + comments[0]);
#print("filtered: "+ str(filtered_comment[0]))

for indi_comment in filtered_comment:
    # remove words that appear only once
    for word in indi_comment:
        frequency[word] +=1;



In [12]:
from pprint import pprint
#pprint(frequency)
filtered_comment = [[token for token in indi_comment if frequency[token] > 1]
        for indi_comment in filtered_comment]

In [13]:
#filtered_comment
type(filtered_comment[0])

list

In [14]:
dictionary = corpora.Dictionary(filtered_comment)
#dictionary.save('ford.dict')

corpus = [dictionary.doc2bow(indi_comment) for indi_comment in filtered_comment]
#corpora.MmCorpus.serialize('/tmp/ford.mm', corpus)

#corpus

In [15]:
#print(dictionary.token2id)

In [16]:
#corpus

In [17]:
lsi = models.LsiModel(corpus, id2word = dictionary, num_topics = 50, power_iters = 3)

lsi.print_topics(10)
#lda = models.LdaModel(corpus, id2word = dictionary, num_topics = 450)

#hdp = models.HdpModel(corpus, id2word = dictionary)


[u'0.780*"" + 0.236*"dr" + 0.138*"t" + 0.135*"doctor" + 0.103*"s" + 0.101*"be" + 0.097*"time" + 0.095*"an" + 0.087*"all" + 0.084*"about"',
 u'0.623*"" + -0.291*"dr" + -0.180*"t" + -0.164*"doctor" + -0.138*"s" + -0.138*"be" + -0.133*"time" + -0.119*"an" + -0.113*"when" + -0.112*"about"',
 u'-0.773*"dr" + 0.278*"t" + -0.113*"has" + -0.109*"staff" + 0.104*"get" + 0.091*"if" + 0.091*"there" + 0.090*"be" + -0.090*"great" + -0.085*"recommend"',
 u'-0.812*"doctor" + 0.266*"dr" + 0.221*"t" + -0.125*"staff" + -0.124*"great" + -0.117*"time" + -0.104*"has" + 0.092*"surgery" + -0.090*"always" + -0.075*"office"',
 u'-0.471*"t" + -0.321*"s" + 0.208*"would" + -0.190*"always" + 0.169*"after" + 0.167*"surgery" + 0.162*"did" + 0.160*"told" + 0.154*"doctor" + -0.136*"don"',
 u'0.422*"doctor" + 0.395*"t" + -0.341*"staff" + -0.299*"time" + -0.284*"office" + 0.241*"dr" + -0.168*"all" + -0.154*"always" + -0.154*"were" + -0.133*"their"',
 u'0.456*"office" + 0.376*"t" + 0.248*"staff" + -0.248*"has" + -0.207*"c

In [18]:
#search_template = widgets.Text(description='Search Template: ', width = 200);                         
print(lsi)
#display(search_template)

LsiModel(num_terms=26745, num_topics=50, decay=1.0, chunksize=20000)


In [19]:
#search_list = search_template.value.split(",");

#print(search_list)

In [20]:
#training_set = pd.read_excel("target_practice_2.xls", 'Sheet1')
training_set = pd.read_csv("/Users/zzhao/Documents/Anthony_code_and_review/a894359.csv")


#training_dict = training_set.set_index('topics_list').to_dict()
training_set

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,what_topics_is_this_review_about,what_topics_is_this_review_about:confidence,comment,id,industry,normalized_rating,what_topics_is_this_review_about_gold
0,929645031,False,finalized,6,4/9/2016 18:11:00,NaN,NaN,Do to my age and kidney function I had hard ti...,SURVEY_e13e8a5544918fee59d70318f973f486_51e150...,NaN,0.8,NaN
1,929645032,False,finalized,5,4/8/2016 22:29:21,bedside_manner,0.6217,Great Dr. !!!\nNever made me feel crazy when I...,SURVEY_d744de84a3f817ed72fb4dd343ac96ea_a85aec...,NaN,1.0,NaN
2,929645033,False,finalized,4,4/8/2016 22:45:45,valuepricebilling_issues,0.7696,"Doctor does bloodwork and bills as $80 ""surger...",VITALS.II1vRBB-0i1O-ejmhdSkcg.6x-9Gr9Xw24XEmVS...,NaN,0.2,NaN
3,929645034,False,finalized,3,4/8/2016 22:22:14,other,1.0,Everything has turned out perfectly. I love my...,SURVEY_100ff08b310cdd59dc5c48b5ff51c099_aa49df...,NaN,1.0,NaN
4,929645035,False,finalized,4,4/8/2016 22:45:45,doctor_competence,1.0,MY HUSBAND HAD SURGERY 5 years ago done by Dr....,VITALS.gZ1_elzRAnjSeU8T_Ru6Zg.ksUFMMmHg1Pialji...,NaN,0.2,NaN
5,929645036,False,finalized,4,4/8/2016 22:25:24,doctor_competence\nbedside_manner,0.7622\n0.4798,We encountered this so-called doctor after our...,VITALS.VfIJcM0l6B7WYqUu00OpAg.qmTCaeQb3DHl1k0N...,NaN,0.2,NaN
6,929645037,False,finalized,6,4/9/2016 18:41:11,bedside_manner\ndoctor_competence,0.6703\n0.4965,Denise started with me & was awesome! Erica w...,SURVEY_7b16fd4c8391892a9015aac2153f4f46_9b2ed9...,NaN,1.0,NaN
7,929645038,False,finalized,3,4/8/2016 22:16:41,valuepricebilling_issues,0.669,BEWARE!! Do not give these people any money up...,VITALS.hbzNhLAdtbO1irtdScDSkA.x8vK6RvC19Jri44F...,NaN,0.2,NaN
8,929645039,False,finalized,3,4/8/2016 22:10:51,bedside_manner,1.0,Very efficient and friendly practice! Dr. Mitt...,SURVEY_9476260730ec6668c2307a8bc511ce78_b103b7...,NaN,1.0,NaN
9,929645040,False,finalized,5,4/8/2016 22:19:15,staff,0.7954,"As uncomfortable as I'm feeling, these employe...",SURVEY_100ff08b310cdd59dc5c48b5ff51c099_5212b4...,NaN,1.0,NaN


In [34]:
#label_first = training_set['Answer.label'].values.tolist();
#label_second = training_set['Answer.v'].values.tolist();

label_first = training_set['what_topics_is_this_review_about'].values.tolist();
input_comm = training_set['comment'].values.tolist();
search_comment = [];
li = [];


#print(type(str(label[4]).split(',')))
j = 0;
#for la in label_first:
#    if "cleanliness" in la and "bedside_manner" not in la and "doctor_competence" not in la and "staff_professionalism" not in la:
#        print(la);
#        #if(len(str(la).split(',')) == 2):
#        search_comment.extend(input_comm[j].lower().split());
#        li.append(j);
#    j+=1;
    
#print(type(label_first[0]))
while j < len(label_first):
#    print(type(label_first[j]))
#    print(label_first[j])
    if "medical_history" in str(label_first[j]):
        print("first: " + label_first[j]);
        search_comment.extend(input_comm[j].lower().split());
        li.append(j);
    j+=1;

print(search_comment)
#print(label[li[2]])


first: doctor_competence
medical_history
first: medical_history
bedside_manner
first: medical_history
bedside_manner
first: doctor_competence
medical_history
first: medical_history
first: bedside_manner
medical_history
first: doctor_competence
medical_history
first: likely_to_recommend
medical_history
doctor_competence
first: doctor_competence
medical_history
['beware!', 'avoid', 'this', 'doctor', 'if', 'at', 'all', 'possible.', 'does', 'not', 'care', 'about', 'your', 'past', 'history', 'of', 'health', 'care.', 'only', 'concerned', 'with', 'textbook', 'ucsd', 'procedures.', 'does', 'not', 'give', 'personal', 'attention', 'to', 'your', 'specific', 'needs', 'dr', 'mehyar', 'hashem', 'lied', 'about', 'medical', 'information', 'in', 'my', 'chart', 'and', 'would', 'not', 'listen', 'to', 'me.', 'he', 'became', 'upset', 'when', 'i', 'asked', 'questions', 'and', 'discharged', 'me', 'from', 'the', 'hospital', 'too', 'early,', 'only', 'for', 'me', 'to', 'be', 'rushed', 'back', 'to', 'er.', 'he',

In [35]:

#search_list = ["pushy", "hassle", "pressured", "pressure", "tactic", "stressful", "upsell"];


#keyword = str(search_list[random.randrange(0, len(search_list))]).strip();
#search_comment = [];
#i = -1;
#for indi_list in filtered_comment:
#    i +=1;
#    if keyword in indi_list:
#        search_comment = indi_list;
#        break;
        
#print(keyword)
#print(search_comment);
#print(comment_list[i]);
doc = dictionary.doc2bow(search_comment);
#print(doc)
#print(search_comment)
# lsi = latent semantic indexing
vec_lsi = lsi[doc]

#print(dictionary)
#vec_lda = lda[doc]

#vec_hdp = hdp[doc]
#type(doc)
pprint(vec_lsi)

[(0, 13.848431162873149),
 (1, -17.596655787091958),
 (2, 2.6473136111203917),
 (3, 0.55757821812896591),
 (4, 8.9828000367635035),
 (5, -0.35193066295066811),
 (6, -5.3385195678916775),
 (7, -1.6852920346009892),
 (8, -1.2309870941090684),
 (9, 2.380770517499613),
 (10, 0.52000339317518451),
 (11, 3.0881074003723388),
 (12, -1.0696162463735739),
 (13, 2.3416845145664178),
 (14, -0.49502053413363911),
 (15, 0.012403408506141016),
 (16, -1.1671311549452092),
 (17, 0.31528140967815593),
 (18, 2.2948194036050005),
 (19, 3.1652046033553445),
 (20, 1.9211013977485172),
 (21, -1.1874366705533816),
 (22, -0.41733045034365213),
 (23, 0.66790779105658105),
 (24, -0.92129209212836372),
 (25, -1.6017512410983226),
 (26, 4.764765605409103),
 (27, 0.80604772926841017),
 (28, -0.0036602170020645303),
 (29, 0.59761611749088683),
 (30, 0.93886441476829585),
 (31, -0.3945594783460214),
 (32, 0.28538188963880079),
 (33, -0.9942514021648089),
 (34, -1.2254780958855269),
 (35, -2.0744483610889879),
 (36, 

In [36]:
index = similarities.MatrixSimilarity(lsi[corpus])

#index_lda = similarities.MatrixSimilarity(lda[corpus])

#index_hdp = similarities.MatrixSimilarity(hdp[corpus])


In [37]:
type(index)

gensim.similarities.docsim.MatrixSimilarity

In [38]:
#perform similarity query against the corpus
sims = index[vec_lsi]

#sims_lda = index_lda[vec_lda]

#print(comment_list[0])


#sims_hdp = index_hdp[vec_hdp]
#pprint(doc[1])
#pprint(dictionary.get(522))


In [39]:
#print(list(enumerate(sims)))
#check because there are way too many similar ones if count ~1 are very similar

In [40]:
# sort by descending order
sims = sorted(enumerate(sims), key = lambda item: -item[1])
#sims_lda = sorted(enumerate(sims_lda), key = lambda item: -item[1])
#sims_hdp = sorted(enumerate(sims_hdp), key = lambda item: -item[1])

print(sims)
#print(sims_lda)
#print(sims_hdp)

[(37643, 0.79894423), (31318, 0.7986902), (37611, 0.7902844), (39060, 0.77752584), (24324, 0.77590579), (34347, 0.77138323), (14135, 0.77134359), (48304, 0.76929408), (27738, 0.76356256), (40073, 0.76335287), (18441, 0.76086545), (26365, 0.75765705), (6217, 0.7557736), (22020, 0.75555944), (800, 0.75386912), (4496, 0.75359166), (33431, 0.75175267), (48688, 0.74990249), (49617, 0.74949133), (39223, 0.74534225), (11123, 0.74444687), (1154, 0.74382854), (38600, 0.74290198), (47050, 0.74210685), (29493, 0.74184966), (8316, 0.74139702), (43408, 0.74139702), (14227, 0.74110991), (7532, 0.74074382), (11897, 0.74023885), (21597, 0.73895276), (20979, 0.73838067), (6658, 0.73751414), (38816, 0.73650819), (26542, 0.73494118), (12385, 0.73205155), (10918, 0.73159349), (11116, 0.73085278), (7450, 0.73085088), (27385, 0.73075408), (44377, 0.72882265), (26799, 0.72816682), (40547, 0.72547972), (41904, 0.72512031), (15374, 0.72507018), (22161, 0.724953), (6864, 0.72475547), (42279, 0.72416228), (19796

In [41]:
#top_lsi_list = [];
#top_lda_list = [];
#single_comment = [];
#single_comment_lda = [];

#ini_comment_index = sims[0][0];

#index = 0;
#while index < 50:
#    single_comment = comment_list[sims[index][0]][:];
#    single_comment.append(sims[index][1]);
#    top_lsi_list.append(single_comment);
    
    #single_comment_lda = comment_list[sims_lda[index][0]][:];
    #single_comment_lda.append(sims_lda[index][1]);
    #top_lda_list.append(single_comment_lda);
    #print(comment_list[sims[index][0]]);
    #print(sims[index][1]);
    #print;
#    index += 1;
    
#print(top_lsi_list)
#sorted_top_lsi_list = sorted(top_lsi_list, key=lambda l:l[1]);
#sorted_top_lda_list = sorted(top_lda_list, key=lambda l:l[1]);

#filtered_comment[16876]

In [42]:
index = 0;
id_set = [];
comment_set = [];
similarity_set = [];
while index < 200:
    #print("Similarity: " + str(sims[index][1]));
    #print(comment_list[sims[index][0]] + "\n");
    id_set.append(id_list[sims[index][0]].encode('ascii', 'ignore'));
    comment_set.append(comment_list[sims[index][0]].encode('ascii', 'ignore'));
    similarity_set.append(str(sims[index][1]).encode('ascii', 'ignore'));
    index +=1;

print(comment_set);

#print(filtered_comment[sims[2][0]]);
#print(comment_list[sims[2][0]])

["My 21-year-old daughter just returned from a visit with this doctor hysterical and in tears. Two weeks prior, she had returned from college with a cold she came down with during finals week, which turned worse during a post-grad week at the beaches in Maryland. Since she was going to be flying to Europe in a week and the cold was now 14-days old and producing green, bloody mucus, I advised her to visit the walk-in medi-center on Route 88 in Brick and tell the doctor her history so she could be evaluated for a bacterial infection and prescribed an antibiotic before heading oversees. When she came back, however, she was crying and extremely upset. She said the doctor had berated and belittled her because she told him she did not suffer from seasonal allergies. He kept insisting to her that she, in fact, did suffer from allergies and that she had better start taking Claritan in the fall. When my daughter again tried to explain that she had never suffered from seasonal allergies, he kept

In [43]:
complete_list = list(zip(id_set, comment_set, similarity_set));
complete_list

[('VITALS.7VqWGTT3NLuUpHzMo-JfLQ.ZlZhw2pnrlAeh46dx5Cc3Q',
  "My 21-year-old daughter just returned from a visit with this doctor hysterical and in tears. Two weeks prior, she had returned from college with a cold she came down with during finals week, which turned worse during a post-grad week at the beaches in Maryland. Since she was going to be flying to Europe in a week and the cold was now 14-days old and producing green, bloody mucus, I advised her to visit the walk-in medi-center on Route 88 in Brick and tell the doctor her history so she could be evaluated for a bacterial infection and prescribed an antibiotic before heading oversees. When she came back, however, she was crying and extremely upset. She said the doctor had berated and belittled her because she told him she did not suffer from seasonal allergies. He kept insisting to her that she, in fact, did suffer from allergies and that she had better start taking Claritan in the fall. When my daughter again tried to explain t

In [44]:
df = pd.DataFrame(data = complete_list,  columns=['id', 'comment', 'similarity_score'])
df

,id,comment,similarity_score
0,VITALS.7VqWGTT3NLuUpHzMo-JfLQ.ZlZhw2pnrlAeh46d...,My 21-year-old daughter just returned from a v...,0.798944
1,VITALS.pmhi7-1pvqEQ8U1RTY0_OQ.A8gvX4138ZegSedo...,I decided to try doctor Borrison after Dr. Bel...,0.79869
2,VITALS.2gCXQNf2v0VCxoo7KGGIaw.xzntxetPjI2t1H4Y...,I went to Dr. Schey for the concern of menopau...,0.790284
3,VITALS.88fTqVKnXrVg1t25Bx12Ng.ZQaowVvqlh0kZSq8...,This is the first hand experience of my wife w...,0.777526
4,VITALS.xWFyieyfQ7MaWWq_tZWUYg._uZJL_iU0Of3-KW_...,I highly recommend Dr. Patterson for any woman...,0.775906
5,SURVEY_e053de4abbd651f10d99aee73829e56e_c85b08...,Here is a word to the wise: If you are in incr...,0.771383
6,VITALS.WENL1KK9gjvJWlT8wxN6Vg.Lzy_w0gGm3CxNsgK...,I take a lot of medications due to chronic pai...,0.771344
7,VITALS.UB7q-caPXiJkfCVatOTbcw.xR7rkCuqbdtr2DK5...,"First, the practice at which I had my tubal li...",0.769294
8,RATE_MDS.5bFOgjZceSNhV7VBIZcI3A.TBoWVuEx9QKJcX...,Would give 0 stars if I could! After waiting f...,0.763563
9,VITALS._P0HDHKRdZQd87L7Y4sdcw.1gJhozNhB0I73hiS...,Fourth time here and finally drew the line. Dr...,0.763353


In [45]:
df.to_csv('medical_history.csv')

In [33]:
#init_comment_widget = widgets.Textarea(description="Initial Comment with keyword " + keyword +": ", value=comment_list[i][0], disabled=True, background_color='white')

#pos_neg_comment_selection = widgets.RadioButtons(description="Similarity Result: ", options= ['Positive', 'Neutral','Negative'])
#result_container = widgets.Box(children = [init_comment_widget, pos_neg_comment_selection]);
#display(result_container);




In [49]:
#def println(li):
#    print("rating = " + str(li[1]) + ": ");
#    print("\t" + li[0]);
#    print("Similarity: " + str(li[2]) + "\n");
#    return;

#if pos_neg_comment_selection.value == 'Negative':
#    for li in sorted_top_lsi_list:
#        if li[1] <= 0.3:
#            println(li)
#elif pos_neg_comment_selection.value == 'Neutral':
#    for li in sorted_top_lsi_list:
#        if (li[1] < 0.7 and li[1] > 0.3):
#            println(li)
#elif pos_neg_comment_selection.value == 'Positive':
#    for li in sorted_top_lsi_list:
#        if li[1] >=0.7:
#            println(li)